In [1]:
import $file.common
import common._

Compiling /Users/jserrano/Documents/tagless-final-tutorial/session1/common.sc

import $file.$     

import common._

# Variation 2. KISS In-memory

In this model, data is stored in in-memory data structures:

In [2]:
case class Country(
    code: String, 
    name: String, 
    capital: Option[Int])

case class City(
    id: Int, 
    name: String, 
    countryCode: String, 
    population: Int)

defined class Country
defined class City

Queries make use of recursive data structures and higher-order functions:

In [3]:
def query(countries: List[Country], 
          cities: List[City]): List[(String, String)] = 
    for {
        country <- countries
        city <- cities if country.capital.exists(_ == city.id)
        if city.population > 8000000
    } yield (country.name, city.name)

defined function query

The same query using an index for cities:

In [4]:
def queryWithIndex(countries: List[Country], 
            cities: Map[Int, City]): List[(String, String)] =
    for {
        Country(_, name, Some(capital)) <- countries
        city <- cities.get(capital).toList
        if city.population > 8000000
    } yield (name, city.name)

defined function queryWithIndex

Test with world database:

In [5]:
import doobie.implicits._

import doobie.implicits._

In [6]:
val realCountries: List[Country] = 
    sql"select code, name, capital from country"
        .query[Country].to[List].transact(xa).unsafeRunSync

val realCities: List[City] = 
    sql"select id, name, countryCode, population from city"
        .query[City].to[List].transact(xa).unsafeRunSync

val realMapCities: Map[Int, City] = 
    Map.from(realCities.map(city => (city.id, city)))

realCountries: List[Country] = List(
  Country("AFG", "Afghanistan", Some(1)),
  Country("NLD", "Netherlands", Some(5)),
  Country("ANT", "Netherlands Antilles", Some(33)),
  Country("ALB", "Albania", Some(34)),
  Country("DZA", "Algeria", Some(35)),
  Country("ASM", "American Samoa", Some(54)),
  Country("AND", "Andorra", Some(55)),
  Country("AGO", "Angola", Some(56)),
  Country("AIA", "Anguilla", Some(62)),
  Country("ATG", "Antigua and Barbuda", Some(63)),
  Country("ARE", "United Arab Emirates", Some(65)),
  Country("ARG", "Argentina", Some(69)),
  Country("ARM", "Armenia", Some(126)),
  Country("ABW", "Aruba", Some(129)),
  Country("AUS", "Australia", Some(135)),
  Country("AZE", "Azerbaijan", Some(144)),
  Country("BHS", "Bahamas", Some(148)),
  Country("BHR", "Bahrain", Some(149)),
  Country("BGD", "Bangladesh", Some(150)),
  Country("BRB", "Barbados", Some(174)),
  Country("BEL", "Belgium", Some(179)),
  Country("BLZ", "Belize", Some(185)),
  Country("BEN", "Benin", Some(187))

In [59]:
query(realCountries, realCities).timed.millis
queryWithIndex(realCountries, realMapCities).timed.nanos

28 millis
274602 nanos


res58_0: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)
res58_1: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)

In [60]:
query(realCountries, realCities).timed(100).millis

28 millis


res59: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)